In [23]:
from Imports import *
from Preprocessing import *

In [24]:
gpuAcceleration = False


In [ ]:
feature_dimension=8
reps=5 #4 is a good number. 10 overfits.

# Data paths
TRAIN_FILE = '../Data/X_train_scaled.csv'
TEST_FILE = '../Data/X_test_scaled.csv'
LABEL_TRAIN_FILE = '../Data/2025-Quantathon-Tornado-Q-training_data-640-examples.xlsx'
LABEL_TEST_FILE = '../Data/2025-Quantum-Tornado-Q-test_data-200-examples.xlsx'
SAVE_LOCATION = "../kernels/"

df_train = pd.read_csv(TRAIN_FILE)
df_test = pd.read_csv(TEST_FILE)

df_label_train_class = pd.read_excel(LABEL_TRAIN_FILE)["ef_class"]

df_label_test_class = pd.read_excel(LABEL_TEST_FILE)["ef_class"]

df_label_train_class, df_train = removeWeakTornados(df_label_train_class, df_train)
df_label_test_class, df_test = removeWeakTornados(df_label_test_class, df_test)

# Renormalize Data
df_train = df_train.drop(df_train.columns[-1], axis = 1) 
df_train = np.tanh(df_train)
df_test = df_test.drop(df_test.columns[-1], axis = 1)
df_test = np.tanh(df_test)

print(f"✓ Training data loaded: {df_train.shape[0]} rows, {df_train.shape[1]} columns")
print(f"✓ Test data loaded: {df_test.shape[0]} rows, {df_test.shape[1]} columns")

print(df_label_train_class.value_counts())
#This training data is imbalanced; I need to 

✓ Training data loaded: 69 rows, 8 columns
✓ Test data loaded: 21 rows, 8 columns
ef_class
2    50
3    19
Name: count, dtype: int64


In [26]:
#Dict of which Encodings/Kernels to compute.
gammas = [50, 45, 40, 35, 30]

encodingSettings = {
    "zzEncoding":True,
    "zzEncoding_full_entangle":True,
    "pauli_feature_map_full_entangle":True
}

classicalKernelSettings = {
    "rbf":True,
    "polynomial":False,
}


In [27]:
#Data Encoding - Different Methods to try. 
#Make this a dict? Then settings choose which to include in dict. Makes training/fitting/faster
feature_maps = {
#basisEncoding = ,
#amplitudeEncoding = ,
#angleEncoding = ,
#phaseEncoding = ,
#denseAngleEncoding = ,
"zzEncoding": zz_feature_map(feature_dimension=feature_dimension, reps=reps, entanglement="linear"),
"zzEncoding_full_entangle": zz_feature_map(feature_dimension=feature_dimension, reps=reps, entanglement="full"),
"pauli_feature_map_full_entangle": pauli_feature_map(feature_dimension=feature_dimension, reps=reps, entanglement="full")
}

#what does varying reps do? Can I show a number on how varying reps changes things?
#Print/store data like circuit depth!
#Try different feature maps! Which is best?

#Different inner products?

In [28]:
for encoding in feature_maps:
    if encodingSettings[encoding]:
        print(f"computing kernel for {encoding} feature map")
        quantum_kernel = FidelityStatevectorKernel(feature_map=feature_maps[encoding])
        quantum_kernel_train = quantum_kernel.evaluate(df_train)
        quantum_kernel_test = quantum_kernel.evaluate(df_test, df_train)
        np.save(f'./kernels/{encoding}_kernel_train.npy', quantum_kernel_train)
        np.save(f'./kernels/{encoding}_kernel_test.npy', quantum_kernel_test)

computing kernel for zzEncoding feature map
computing kernel for zzEncoding_full_entangle feature map
computing kernel for pauli_feature_map_full_entangle feature map


In [29]:
rbf_kernel_train = {}
rbf_kernel_test = {}

if classicalKernelSettings["rbf"]:
    for gamma in gammas:
        rbf_kernel_train[gamma] = rbf_kernel(df_train, gamma=gamma)
        rbf_kernel_test[gamma] = rbf_kernel(df_test, df_train, gamma=gamma)
        np.save(f'./kernels/rbf_gamma={gamma}_kernel_train.npy', rbf_kernel_train)
        np.save(f'./kernels/rbf_gamma={gamma}_kernel_test.npy', rbf_kernel_test)